In [9]:
def filter_by_moneyness(df, pct_cutoff=0.2):
    crit1 = (1-pct_cutoff)*df.Strike < df.Underlying
    crit2 = df.Underlying< (1+pct_cutoff)*df.Strike
    return (df.loc[crit1 & crit2].reset_index(drop=True))            

def options_chain(symbol):
    import datetime
    import re
    import pandas as pd
    import QuantLib as ql
    import pandas as pd
    import numpy as np
    import yfinance as yf
    from yahoo_fin.stock_info import get_quote_table
    from pylab import plt
    plt.style.use('dark_background')
    import pandas as pd

    info = get_quote_table(symbol)
    current_price = info["Quote Price"]
    tk = yf.Ticker(symbol)
    exps = tk.options
    options = pd.DataFrame()
    for e in exps:
        opt = tk.option_chain(e)
        opt = pd.DataFrame().append(opt.calls).append(opt.puts)
        opt['expirationDate'] = e
        options = options.append(opt, ignore_index=True)
    options['expirationDate'] = pd.to_datetime(options['expirationDate']) + datetime.timedelta(days = 1)
    options['yte'] = (options['expirationDate'] - datetime.datetime.today()).dt.days / 365
    options['dte'] = (options['expirationDate'] - datetime.datetime.today()).dt.days
    options['CALL'] = options['contractSymbol'].str[4:].apply(
        lambda x: "C" in x)
    options[['bid', 'ask', 'strike']] = options[['bid', 'ask', 'strike']].apply(pd.to_numeric)
    options['midpoint'] = (options['bid'] + options['ask']) / 2 
    options['spread'] =  (options['ask'] - options['bid'])
    options['spread_pct'] = (options['ask'] - options['bid'])/options['ask'] 
    options['Underlying'] = current_price 
    options = options.drop(columns = ['contractSize', 'currency', 'change', 'percentChange', 'lastTradeDate'])
    return pd.DataFrame({
            "Underlying": current_price,
            "Ticker": symbol,
            "Expiry": options["expirationDate"],
            "YTE": options["yte"],
            "DTE": options["dte"],
            "Call": options["CALL"],
            "IV": options["impliedVolatility"],        
            "Strike": options["strike"],
            "Last": options["lastPrice"],
            "Bid": options["bid"],
            "Ask": options["ask"],
            "Midpoint": options['midpoint'],
            "Spread": options['spread'],
            "Spread_Pct": options['spread_pct'],
            "AKA": options["contractSymbol"]})

In [10]:
df=options_chain("AAPL")
df=filter_by_moneyness(df)

Example

Vanilla Call Option


In [3]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [147]:
def filter_by_moneyness(df, pct_cutoff=0.2):
    crit1 = (1-pct_cutoff)*df.Strike < df.Underlying
    crit2 = df.Underlying< (1+pct_cutoff)*df.Strike
    return (df.loc[crit1 & crit2].reset_index(drop=True))            

def know_your_options(ticker, option="Call"):
    import datetime
    import re
    import utils 
    import QuantLib as ql 
    import pandas as pd
    import yfinance as yf
    import numpy as np
    import matplotlib.pyplot as plt
    from yahoo_fin.stock_info import get_quote_table

    import warnings
    plt.style.use('dark_background')
    warnings.simplefilter(action='ignore', category=FutureWarning)
                
    info = get_quote_table(ticker)
    current_price = info["Quote Price"]
    yield_re = re.compile(r"\((?P<value>(\d+\.\d+))%\)")
    try:
        dividend_rate = float(yield_re.search(info["Forward Dividend & Yield"])["value"])
    except (KeyError, ValueError, TypeError):
        dividend_rate = 0.0      
        
    def create_call(row):
        
        calculation_date = ql.Date.todaysDate()
        ql.Settings.instance().evaluationDate = calculation_date

        risk_free_rate = 0.001
        day_count = ql.Actual365Fixed()
        settlement = calculation_date
        calendar = ql.UnitedStates()

        exercise = ql.AmericanExercise(settlement, ql.Date(expiration.day, expiration.month, expiration.year))
        payoff = ql.PlainVanillaPayoff(ql.Option.Call, row["strike"])
        american_option = ql.VanillaOption(payoff,exercise)
        
        spot_handle = ql.QuoteHandle(ql.SimpleQuote(current_price))
        flat_ts = ql.YieldTermStructureHandle(ql.FlatForward(calculation_date, risk_free_rate, day_count))
        dividend_yield = ql.YieldTermStructureHandle(ql.FlatForward(calculation_date, dividend_rate, day_count))
        flat_vol_ts = ql.BlackVolTermStructureHandle(ql.BlackConstantVol(calculation_date, calendar, row["impliedVolatility"], day_count))
        bsm_process = ql.BlackScholesMertonProcess(spot_handle, dividend_yield, flat_ts, flat_vol_ts)
            
        american_option.setPricingEngine(ql.FdBlackScholesVanillaEngine(bsm_process, 2000, 200))

        return {"Underlying": current_price,
                "Ticker": ticker,
                "YTE": row["yte"],
                "DTE": row["dte"],
                "IV": row["impliedVolatility"],
                "Strike": row["strike"],
                "Last": row["lastPrice"],
                "Bid": row["bid"],
                "Ask": row["ask"],
                "NPV": american_option.NPV(),
                "Delta": american_option.delta(),
                "Gamma": american_option.gamma(),
                "Theta": american_option.theta() / 365,
                "AKA": row["contractSymbol"]}        
        
    def create_put(row):

        calculation_date = ql.Date.todaysDate()
        ql.Settings.instance().evaluationDate = calculation_date

        risk_free_rate = 0.001
        day_count = ql.Actual365Fixed()
        settlement = calculation_date
        calendar = ql.UnitedStates()
        
        exercise = ql.AmericanExercise(settlement, ql.Date(expiration.day, expiration.month, expiration.year))
        payoff = ql.PlainVanillaPayoff(ql.Option.Put, row["strike"])
        american_option = ql.VanillaOption(payoff,exercise)
        
        spot_handle = ql.QuoteHandle(ql.SimpleQuote(current_price))
        flat_ts = ql.YieldTermStructureHandle(ql.FlatForward(calculation_date, risk_free_rate, day_count))
        dividend_yield = ql.YieldTermStructureHandle(ql.FlatForward(calculation_date, dividend_rate, day_count))
        flat_vol_ts = ql.BlackVolTermStructureHandle(ql.BlackConstantVol(calculation_date, calendar, row["impliedVolatility"], day_count))
        bsm_process = ql.BlackScholesMertonProcess(spot_handle, dividend_yield, flat_ts, flat_vol_ts)
            
        american_option.setPricingEngine(ql.FdBlackScholesVanillaEngine(bsm_process, 2000, 200))
        

        return {"Underlying": current_price,
                "Ticker": ticker,
                "YTE": row["yte"],
                "DTE": row["dte"],
                "IV": row["impliedVolatility"],
                "Strike": row["strike"],
                "Last": row["lastPrice"],
                "Bid": row["bid"],
                "Ask": row["ask"],
                "NPV": american_option.NPV(),
                "Delta": american_option.delta(),
                "Gamma": american_option.gamma(),
                "Theta": american_option.theta() / 365,
                "AKA": row["contractSymbol"]}                  
        
    options_= pd.DataFrame()       
    
    if option == "Call":
        
        tk = yf.Ticker(ticker)
        exps = tk.options
        
        for e in exps:
            opt = tk.option_chain(e)
            calls = pd.DataFrame().append(opt.calls)
            calls['expirationDate'] = e
            expiration =  pd.to_datetime(e) + datetime.timedelta(days = 1)
            calls['expirationDate'] = expiration
            calls['yte'] = (calls['expirationDate'] - datetime.datetime.today()).dt.days / 365
            calls['dte'] = (calls['expirationDate'] - datetime.datetime.today()).dt.days
            calls[['bid', 'ask', 'strike']] = calls[['bid', 'ask', 'strike']].apply(pd.to_numeric)
            calls = calls.drop(columns = ['contractSize', 'currency', 'change', 'percentChange', 'lastTradeDate'])
            options = calls.apply(create_call, axis=1, result_type="expand")
            options_ = options_.append(options, ignore_index=True) 
    else:    
        
        tk = yf.Ticker(ticker)
        exps = tk.options
        
        for e in exps:
            opt = tk.option_chain(e)
            puts = pd.DataFrame().append(opt.calls)
            puts['expirationDate'] = e
            expiration =  pd.to_datetime(e) + datetime.timedelta(days = 1)
            puts['expirationDate'] = expiration
            puts['yte'] = (puts['expirationDate'] - datetime.datetime.today()).dt.days / 365
            puts['dte'] = (puts['expirationDate'] - datetime.datetime.today()).dt.days
            puts[['bid', 'ask', 'strike']] = puts[['bid', 'ask', 'strike']].apply(pd.to_numeric)
            puts = puts.drop(columns = ['contractSize', 'currency', 'change', 'percentChange', 'lastTradeDate'])
            options = puts.apply(create_put, axis=1, result_type="expand")
            options_ = options_.append(options, ignore_index=True) 
    return  options_

In [148]:
options = know_your_options("AAPL", option="Call")

In [154]:
options_filtered = filter_by_moneyness(options)

In [155]:
options_filtered

,Underlying,Ticker,YTE,DTE,IV,Strike,Last,Bid,Ask,NPV,Delta,Gamma,Theta,AKA
0,119.989998,AAPL,0.013699,5,0.539067,100.0,20.00,19.90,20.00,19.989998,1.000000,6.714605e-09,0.000000e+00,AAPL210326C00100000
1,119.989998,AAPL,0.013699,5,0.570317,105.0,15.38,14.90,15.10,14.989998,1.000000,-2.953935e-09,-1.236452e-11,AAPL210326C00105000
2,119.989998,AAPL,0.013699,5,0.484380,106.0,14.10,13.90,14.05,13.989998,1.000000,-2.098697e-09,-3.947460e-14,AAPL210326C00106000
3,119.989998,AAPL,0.013699,5,0.503911,108.0,12.17,11.95,12.15,11.989998,1.000000,-1.708704e-07,-1.498937e-09,AAPL210326C00108000
4,119.989998,AAPL,0.013699,5,0.468755,109.0,11.40,11.00,11.15,10.989998,1.000000,-1.658109e-08,8.997104e-10,AAPL210326C00109000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
364,119.989998,AAPL,1.989041,726,0.355551,125.0,21.79,21.60,21.85,2.511237,0.246740,2.214814e-02,-8.850005e-06,AAPL230317C00125000
365,119.989998,AAPL,1.989041,726,0.355201,130.0,20.00,19.90,20.05,1.636311,0.161113,1.448078e-02,-8.563235e-06,AAPL230317C00130000
366,119.989998,AAPL,1.989041,726,0.356452,135.0,18.45,18.35,18.50,1.106825,0.108317,9.666833e-03,-9.311522e-06,AAPL230317C00135000
367,119.989998,AAPL,1.989041,726,0.357260,140.0,16.93,16.85,17.05,0.756824,0.073788,6.577812e-03,-8.705975e-06,AAPL230317C00140000


In [143]:
import pandas as pd

def custom_describe(df, nidx=3, nfeats=20):
    ''' Concat transposed topN rows, numerical desc & dtypes '''

    print(df.shape)
    nrows = df.shape[0]
    
    rndidx = np.random.randint(0,len(df),nidx)
    dfdesc = df.describe().T

    for col in ['mean','std']:
        dfdesc[col] = dfdesc[col].apply(lambda x: np.round(x,2))
 
    dfout = pd.concat((df.iloc[rndidx].T, dfdesc, df.dtypes), axis=1, join='outer')
    dfout = dfout.loc[df.columns.values]
    dfout.rename(columns={0:'dtype'}, inplace=True)
    
    # add count nonNAN, min, max for string cols
    nan_sum = df.isnull().sum()
    dfout['count'] = nrows - nan_sum
    dfout['min'] = df.min().apply(lambda x: x[:6] if type(x) == str else x)
    dfout['max'] = df.max().apply(lambda x: x[:6] if type(x) == str else x)
    dfout['nunique'] = df.apply(pd.Series.nunique)
    dfout['nan_count'] = nan_sum
    dfout['pct_nan'] = nan_sum / nrows
    
    return dfout.iloc[:nfeats, :]

In [144]:
custom_describe(options)

(369, 14)


,309,65,257,count,mean,std,min,25%,50%,75%,max,dtype,nunique,nan_count,pct_nan
Underlying,119.99,119.99,119.99,369,119.99,0.00,119.99,119.989998,119.989998,119.989998,119.99,float64,1,0,0.0
Ticker,AAPL,AAPL,AAPL,369,NaN,NaN,AAPL,NaN,NaN,NaN,AAPL,object,1,0,0.0
YTE,0.838356,0.030137,0.320548,369,0.39,0.55,0.0136986,0.052055,0.090411,0.569863,1.98904,float64,16,0,0.0
DTE,306,11,117,369,143.70,202.38,5,19.000000,33.000000,208.000000,726,int64,16,0,0.0
IV,0.352851,0.321784,0.355597,369,0.37,0.05,0.312995,0.348365,0.353522,0.369879,0.601566,float64,281,0,0.0
Strike,140,126,135,369,122.46,12.90,100,112.000000,122.000000,133.000000,149,float64,70,0,0.0
Last,8.53,0.87,4.5,369,9.38,8.59,0.01,1.460000,7.200000,16.190000,32.9,float64,304,0,0.0
Bid,8.35,0.87,4.4,369,9.13,8.37,0,1.370000,7.100000,15.450000,32.75,float64,284,0,0.0
Ask,8.55,0.89,4.45,369,9.26,8.45,0.02,1.450000,7.250000,15.650000,32.95,float64,291,0,0.0
NPV,0.682249,0.4213,0.843195,369,4.96,5.75,0.00405111,0.465056,2.382988,7.989998,19.99,float64,369,0,0.0
